In [139]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler , LabelEncoder
import pickle

In [140]:
## Load the dataset
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [141]:
## Drop columns 
data = data.drop(['RowNumber','CustomerId','Surname'], axis=1)

In [142]:
# Encode Categorical variables
label_encoder_gender  = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [143]:
## One hot encoding
from sklearn.preprocessing import OneHotEncoder

# one hot encoding
ohe_geo = OneHotEncoder(sparse_output=True)
geo_encoder  = ohe_geo.fit_transform(data[['Geography']])
geo_encoder

<10000x3 sparse matrix of type '<class 'numpy.float64'>'
	with 10000 stored elements in Compressed Sparse Row format>

In [144]:
geo_encoder.shape

(10000, 3)

In [145]:
ohe_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [146]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [147]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns=ohe_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [148]:
## Combine the OHE with data
data = pd.concat([geo_encoded_df, data], axis=1)
data

,Geography_France,Geography_Germany,Geography_Spain,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1.0,0.0,0.0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,0.0,0.0,1.0,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,1.0,0.0,0.0,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,1.0,0.0,0.0,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,0.0,0.0,1.0,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.0,0.0,0.0,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,1.0,0.0,0.0,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,1.0,0.0,0.0,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,0.0,1.0,0.0,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [149]:
data.drop(columns=['Geography'], inplace=True)

In [150]:
data

,Geography_France,Geography_Germany,Geography_Spain,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1.0,0.0,0.0,619,0,42,2,0.00,1,1,1,101348.88,1
1,0.0,0.0,1.0,608,0,41,1,83807.86,1,0,1,112542.58,0
2,1.0,0.0,0.0,502,0,42,8,159660.80,3,1,0,113931.57,1
3,1.0,0.0,0.0,699,0,39,1,0.00,2,0,0,93826.63,0
4,0.0,0.0,1.0,850,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.0,0.0,0.0,771,1,39,5,0.00,2,1,0,96270.64,0
9996,1.0,0.0,0.0,516,1,35,10,57369.61,1,1,1,101699.77,0
9997,1.0,0.0,0.0,709,0,36,7,0.00,1,0,1,42085.58,1
9998,0.0,1.0,0.0,772,1,42,3,75075.31,2,1,0,92888.52,1


In [151]:
## dave the encoders and scalers

with open('label_encoder_gender.pkl', 'wb') as f:
    pickle.dump(label_encoder_gender, f)

with open('geo_encoder.pkl', 'wb') as f:
    pickle.dump(ohe_geo, f)

In [152]:
## dicide the data into indep and dep data 
X = data.drop(['Exited'], axis=1)
y = data['Exited']

## split the data into train and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Scale the features 
# we scale the reason is to make the model more robust to outliers
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [153]:
# save scaler to pickle
with open('scaler_pickle.pkl', 'wb') as f:
    pickle.dump(sc, f)

# ANN Implementation

In [154]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping , TensorBoard
import datetime

In [155]:
X_train.shape

(8000, 12)

In [156]:
X_train.shape[1]

12

In [157]:
model = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), ## Hidden layer  1 with 64 neurons
    Dense(32,activation='relu'), ## Hidden layer  2 with 32 neurons
    Dense(1,activation='sigmoid') ## Output layer with 1 neuron
])

/home/omkar/anaconda3/envs/mlenv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [158]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [159]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()

In [160]:
## compile the model 
model.compile(loss=loss, optimizer=opt, metrics=['accuracy'])

In [161]:
## set up the tensorboard
log_dir = './logs/fit/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [162]:
## Set up Early Stopping 
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [163]:
## Training the model
history = model.fit(X_train, y_train, epochs=100, 
                    callbacks=[tensorflow_callback,early_stopping_callback],
                    validation_data=(X_test, y_test))

Epoch 1/100


250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.8154 - loss: 0.4320 - val_accuracy: 0.8575 - val_loss: 0.3542
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8601 - loss: 0.3424 - val_accuracy: 0.8595 - val_loss: 0.3477
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8591 - loss: 0.3402 - val_accuracy: 0.8550 - val_loss: 0.3529
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8571 - loss: 0.3462 - val_accuracy: 0.8525 - val_loss: 0.3530
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8636 - loss: 0.3392 - val_accuracy: 0.8615 - val_loss: 0.3502
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8746 - loss: 0.3214 - val_accuracy: 0.8575 - val_loss: 0.3436
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8633 - loss: 0.3343 - val_accuracy: 0.8600 - val_loss: 0.3394
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8671 - loss: 0.3259 - val_accuracy: 0.8635

In [164]:
model.save('model.h5')

In [165]:
## load Tensorboard Extension 

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [166]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 102127), started 0:33:30 ago. (Use '!kill 102127' to kill it.)

In [167]:
## load the pickle file
